In [10]:
whales = {
    "0xB604f2d512EaA32E06F1ac40362bC9157cE5Da96",
    "0xB604f2d512EaA32E06F1ac40362bC9157cE5Da96"
}

Working prototype: uses etherscan api to request different actions

In [9]:
import os
import requests
import json
from dotenv import load_dotenv

# --- 1. Setup ---

# Load environment variables from .env file
load_dotenv()
API_KEY = os.getenv("ETHERSCAN_API_KEY")

if not API_KEY:
    raise Exception("ETHERSCAN_API_KEY not found in .env file. Please add it.")

# etherscan v2 API endpoint URL
SCAN_URL = "https://api.etherscan.io/v2/api?chainid=8453"


WHALE_ADDRESS = "0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13"

# --- 2. Construct and Send API Request ---

# Parameters for the API request
# We are asking for the ERC-20 token transfers for the specified address

params = {
    "module": "account",
    "action": "tokentx",
    "address": WHALE_ADDRESS,
    "page": 1,
    "offset": 100,  # Get the 100 most recent transactions
    "sort": "desc", # 'desc' for latest transactions first
    "apikey": API_KEY
}

print(f"🔍 Fetching ERC-20 token transfers for address: {WHALE_ADDRESS}")

try:
    # Make the GET request to the Basescan API
    response = requests.get(SCAN_URL, params=params)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

    # --- 3. Process and Save the Data ---
    
    data = response.json()

    # Basescan API returns '1' if successful, '0' if not. Check the status.
    if data["status"] == "1":
        # The actual transaction data is in the 'result' key
        transactions = data["result"]
        print(f"✅ Found {len(transactions)} token transfers.")

        # Define the output filename
        output_filename = "whale_token_transfers.json"

        # Write the raw data to a JSON file
        with open(output_filename, 'w') as f:
            json.dump(transactions, f, indent=4)

        print(f"Successfully saved data to '{output_filename}'")

    else:
        # If status is '0', the 'result' key contains the error message
        print(f"❌ API Error: {data['message']}")
        print(f"   Result: {data['result']}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred with the network request: {e}")
except json.JSONDecodeError:
    print("Failed to decode JSON from the response. The API might be down or returning an invalid format.")



🔍 Fetching ERC-20 token transfers for address: 0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13
✅ Found 100 token transfers.
Successfully saved data to 'whale_token_transfers.json'


simple data structure to count the number of times a given wallet interacts
with different accounts

In [3]:
import json
from collections import defaultdict


def analyze_interactions(json_data):
    '''
    simple data structure to count the number of times a given wallet interacts
    with different accounts
    Args: 
        json str or list
    Returns:
        dict: Dictionary with from_addresses as keys and interaction counts as values
        int: Total number of unique senders
    '''
    # Parse JSON
    if isinstance(json_data, str):
        transfers = json.loads(json_data)
    else:
        transfers = json_data

    #initialize hashmap
    sender_counts = defaultdict(int)

    for transfer in transfers:
        if not isinstance(transfer, dict):
            continue

        from_address = transfer.get("from", "").lower()

        if from_address:
            sender_counts[from_address] += 1
    return dict(sender_counts), len(sender_counts)

def print_interaction_summary(interactions, top_n=10):
    '''
    prints summary of interaction data
    Args:
        interactions (dict): Dictionary with addresses and their interaction counts
        top_n (int): Number of top interactors to show
    '''
    # Add error handling later in case of variable inputs

    total_interactions = sum(interactions.values())
    unique_senders = len(interactions)

    print(f"total transactions: {total_interactions}")
    print(f"Unique sender addresses: {unique_senders}")

    sorted_interactions = sorted(interactions.items(), key=lambda x: x[1], reverse=True)

    print(f"\nTop {min(top_n, len(sorted_interactions))} senders by frequency:")
    for i, (address, count) in enumerate(sorted_interactions[:top_n], 1):
        print(f"{i}. {address}: {count} transactions")

In [4]:
with open('whale_token_transfers.json', 'r') as file:
    token_transfers = json.load(file)

# Analyze interactions
sender_interactions, unique_senders = analyze_interactions(token_transfers)
print_interaction_summary(sender_interactions)

# Look up specific sender (if needed)
specific_address = "0xae2fc483527b8ef99eb5d9b44875f005ba1fae13"
if specific_address.lower() in sender_interactions:
    print(f"\nAddress {specific_address} has sent {sender_interactions[specific_address.lower()]} transactions")

total transactions: 100
Unique sender addresses: 54

Top 10 senders by frequency:
1. 0xd152f549545093347a162dce210e7293f1452150: 15 transactions
2. 0xad3b67bca8935cb510c8d18bd45f0b94f54a968f: 11 transactions
3. 0x0cb5cac087870bdd4db7023191525151f7988dc5: 6 transactions
4. 0x9a26be25ca0da8a7ff5dfbb1aebe587903c2b472: 5 transactions
5. 0xf977814e90da44bfa03b6295a0616a897441acec: 3 transactions
6. 0x3304e22ddaa22bcdc5fca2269b418046ae7b566a: 3 transactions
7. 0x00000688768803bbd44095770895ad27ad6b0d95: 3 transactions
8. 0xd0b53d9277642d899df5c87a3966a349a798f224: 2 transactions
9. 0x80eff5f871f3fcb8767b9b64ac564244751104ca: 2 transactions
10. 0xfaa0b8ca9a4b493f1f749f647ab9d3fdc531048d: 2 transactions


In [3]:
import requests
import json

url = "https://dimensional-cosmological-dew.base-mainnet.quiknode.pro/bab722b969921ea65e90097f3a15404c92e6c635"

payload = json.dumps({
  "method": "eth_getBlockReceipts",
  "params": [
    "latest"
  ],
  "id": 1,
  "jsonrpc": "2.0"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"jsonrpc":"2.0","id":1,"result":[{"blockHash":"0x2ca8a3c37fca77f557ccbe0551b1a381214898bcc4a67c3cabc05cc26a2e5155","blockNumber":"0x1fe3488","contractAddress":null,"cumulativeGasUsed":"0xb44c","depositNonce":"0x1fe348a","depositReceiptVersion":"0x1","effectiveGasPrice":"0x0","from":"0xdeaddeaddeaddeaddeaddeaddeaddeaddead0001","gasUsed":"0xb44c","logs":[],"logsBloom":"0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000","status":"0x1","to":"0x4200000000000000000000000000000000000015","transactionHash":"0x6b035d6997105e16bf0b170ef73

In [11]:
TRANSFER_EVENT_TOPIC = w3.keccak(text="Transfer(address,address,uint256)").to_0x_hex()
print(f"ERC-20 Transfer Event Topic: {TRANSFER_EVENT_TOPIC}")

ERC-20 Transfer Event Topic: 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef


## Data pipeline
Setup and connections

In [19]:
import os
import json
from web3 import Web3
from web3.middleware import ExtraDataToPOAMiddleware
from dotenv import load_dotenv
import requests
from typing import List, Dict, Any, Optional
from datetime import datetime
import time

load_dotenv()
QUICKNODE_URL = os.getenv("QUICKNODE_BASE_URL")
BASESCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")

if not QUICKNODE_URL or not BASESCAN_API_KEY:
    raise Exception("QUICKNODE_BASE_URL and BASESCAN_API_KEY must be set in the .env file.")

# Initialize a Web3 instance with your RPC provider
# Note: We will add the PoA middleware needed for Base in the next step
w3 = Web3(Web3.HTTPProvider(QUICKNODE_URL))

w3.middleware_onion.inject(ExtraDataToPOAMiddleware(), layer=0)

if not w3.is_connected():
    raise Exception("Failed to connect to the Base network via your RPC provider.")

print(f"✅ Successfully connected to Base network (Chain ID: {w3.eth.chain_id})")

✅ Successfully connected to Base network (Chain ID: 8453)


Constants & Cache

In [29]:
TRANSFER_EVENT_TOPIC = w3.keccak(text="Transfer(address,address,uint256)").to_0x_hex()
ERC20_ABI = json.loads('[{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"}]')
COINGECKO_ASSET_PLATFORM_ID = 'base'
# Caches to minimize api calls
TOKEN_METADATA_CACHE: Dict[str, Dict[str, Any]] = {}
PRICE_CACHE: Dict[str, Optional[float]] = {}

In [32]:
# TOKEN_ID_MAP = {
#     "0x4200000000000000000000000000000000000006": "ethereum",  # Wrapped Ether (WETH)
#     "0x833589fcd6edb6e08f4c7c32d4f71b54bda02913": "usd-coin",  # USD Coin (USDC)
#     "0x50c5725949a6f092e06c420be4227906c62a92a5": "dai",       # Dai Stablecoin (DAI)
#     "0x940181a94a35a4569e4529a3cdfb74e38fd98631": "tether",    # Tether (USDT) - Bridged
#     "0x4ed4e862860bed51a9570b96d89af5e1b0efefed": "degen-base" # Degen
# }
token_map_path = "token_contract_mapping.json"
with open(token_map_path, 'r') as f:
    TOKEN_ID_MAP = json.load(f)

In [35]:
print(TOKEN_ID_MAP)

{'0x22aF33FE49fD1Fa80c7149773dDe5890D3c76F3b': {'name': 'BankrCoin', 'symbol': 'BNKR'}, '0x4200000000000000000000000000000000000006': {'name': 'Wrapped Ether', 'symbol': 'WETH'}, '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913': {'name': 'USD Coin', 'symbol': 'USDC'}, '0x940181a94A35A4569E4529A3CDfB74e38FD98631': {'name': 'Aerodrome', 'symbol': 'AERO'}, '0x0b3e328455c4059EEb9e3f84b5543F74E24e7E1b': {'name': 'Virtual Protocol', 'symbol': 'VIRTUAL'}, '0x44551CA46Fa5592bb572E20043f7C3D54c85cAD7': {'name': 'Clanker Index', 'symbol': 'CLX'}, '0x3055913c90Fcc1A6CE9a358911721eEb942013A1': {'name': 'PancakeSwap Token', 'symbol': 'Cake'}, '0x205344EfAd0a46329b752Fb6E33CB6F28d6Db2F4': {'name': 'Conspiracy Guy ', 'symbol': 'CGUY'}, '0x2Dc1C8BE620b95cBA25D78774F716F05B159C8B9': {'name': 'Based Bonk', 'symbol': 'Bonk'}, '0x85E90a5430AF45776548ADB82eE4cD9E33B08077': {'name': 'DINO', 'symbol': 'DINO'}, '0x532f27101965dd16442E59d40670FaF5eBB142E4': {'name': 'Brett', 'symbol': 'BRETT'}, '0x7DA8481207A0e411

maps token contracts to name and symbol, need to make sure file does not get overwritten each time function is called

In [ ]:
def create_token_id_mapping(json_data, output_filepath):
    token_map = {}
    
    for transfer in json_data:
        if not isinstance(transfer, dict):
            continue
    
        contract = transfer.get("tokenContract","")
        name = transfer.get("tokenName","")
        symbol = transfer.get("tokenSymbol")

        if contract and name and symbol:
            token_map[contract] = {
                "name": name,
                "symbol": symbol
            }
        
        with open(output_filepath, 'w') as f:
            json.dump(token_map, f, indent=2)

In [31]:
import json
from collections import defaultdict
from typing import Dict, Tuple, List

def create_token_contract_mapping(json_data: List[dict], output_filepath: str) -> None:
    """
    Creates a mapping of token contracts to their names and symbols from transfer data
    and saves it to a JSON file.
    
    Args:
        json_data (list): List of token transfer objects
        output_filepath (str): Path where the output JSON file will be saved
    """
    # Initialize token contract mapping
    token_map = {}
    
    # Process each transfer
    for transfer in json_data:
        if not isinstance(transfer, dict):
            continue
            
        contract = transfer.get("tokenContract", "")
        name = transfer.get("tokenName", "")
        symbol = transfer.get("tokenSymbol", "")
        
        if contract and name and symbol:
            token_map[contract] = {
                "name": name,
                "symbol": symbol
            }
    
    # Save to JSON file
    with open(output_filepath, 'w') as f:
        json.dump(token_map, f, indent=2)
    
    print(f"Token mapping saved to {output_filepath}")
    print(f"Found {len(token_map)} unique token contracts")

# Example usage
with open('/Users/Lorenzo/Desktop/Work/Obsidian Ridge/Trader Intelligence/blockchain_intel/final_enriched_transfers_block_33606342.json', 'r') as file:
    token_transfers = json.load(file)

# Create the mapping and save to file
create_token_contract_mapping(
    token_transfers, 
    '/Users/Lorenzo/Desktop/Work/Obsidian Ridge/Trader Intelligence/blockchain_intel/token_contract_mapping.json'
)

Token mapping saved to /Users/Lorenzo/Desktop/Work/Obsidian Ridge/Trader Intelligence/blockchain_intel/token_contract_mapping.json
Found 39 unique token contracts


Data fetching

In [ ]:
def get_receipts_for_block(block_number):
    """
    Fetches all transaction receipts for a given block number.
    This uses a custom RPC method often available on enhanced nodes like QuickNode.
    
    Args:
        block_number (int): The block number to fetch receipts for.

    Returns:
        list: A list of transaction receipt objects, or None if an error occurs.
    """
    print(f"\nAttempting to fetch receipts for block: {block_number}...")
    try:
        payload = json.dumps({
            "method": "eth_getBlockReceipts",
            "params": [
                hex(block_number) # "latest"
                ],
            "id": 1,
            "jsonrpc": "2.0"
        })
        headers = {
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", QUICKNODE_URL, headers=headers, data=payload)
        #response = requests.post(QUICKNODE_URL, json=payload)
        response.raise_for_status() # Raise an exception for bad status codes
        
        data = response.json()
        
        if "error" in data:
            print(f"❌ RPC Error: {data['error']['message']}")
            return None
        
        block_info = w3.eth.get_block(block_number)
        
            
        raw = data.get("result", [])
        if isinstance(raw, dict) and "receipts" in raw:
            receipts = raw["receipts"]
        elif isinstance(raw, list):
            receipts = raw
        else:
            receipts = []
        
        print(f"✅ Found {len(receipts)} receipts for block {block_number}.")
        return receipts

    except requests.exceptions.RequestException as e:
        print(f"❌ Network error while fetching receipts: {e}")
        return None
    except json.JSONDecodeError:
        print("❌ Failed to decode JSON from the response.")
        return None

In [33]:
def get_block_with_receipts(block_number: int) -> Optional[Dict[str, Any]]:
    """Fetches block data and all its transaction receipts."""
    print(f"\nAttempting to fetch block and receipts for: {block_number}...")
    try:
        payload = json.dumps({
            "method": "eth_getBlockReceipts",
            "params": [
                hex(block_number) # "latest"
                ],
            "id": 1,
            "jsonrpc": "2.0"
        })
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", QUICKNODE_URL, headers=headers, data=payload)
        response.raise_for_status()
        data = response.json()
        if "error" in data:
            print(f"❌ RPC Error: {data['error']['message']}")
            return None
        
        # We need the block timestamp later, so let's get it now.
        block_info = w3.eth.get_block(block_number)
        result = {"receipts": data.get("result", []), "timestamp": block_info['timestamp']}
        
        print(f"✅ Found {len(result['receipts'])} receipts for block {block_number}.")
        return result
    except Exception as e:
        print(f"❌ Error in get_block_with_receipts: {e}")
        return None

Data parsing logic

In [12]:
def parse_transfers_from_receipts(receipts: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Parses a list of transaction receipts to find and decode ERC-20 Transfer events.
    """
    print(f"\nParsing {len(receipts)} receipts for ERC-20 transfers...")
    decoded_transfers = []

    for receipt in receipts:
        # Each receipt has a 'logs' array
        for log in receipt.get('logs', []):
            # A log's 'topics' array contains indexed event parameters.
            # For an ERC-20 Transfer, topic[0] is the event signature.
            if log['topics'] and log['topics'][0] == TRANSFER_EVENT_TOPIC:
                try:
                    # The 'from' and 'to' addresses are the 2nd and 3rd topics.
                    # They are 32 bytes long, so we slice the last 20 bytes for the address.
                    from_address = Web3.to_checksum_address('0x' + log['topics'][1][-40:])
                    to_address = Web3.to_checksum_address('0x' + log['topics'][2][-40:])
                    
                    log_data = log.get('data', '0x')
                    if log_data == '0x':
                        value = 0
                    else:
                        value = int(log_data, 16)
                    # The 'value' is in the 'data' field, which is not indexed.
                    # It's a hex string representing a uint256.
                    
                    # The address of the token contract that emitted this event
                    token_contract = Web3.to_checksum_address(log['address'])

                    transfer_data = {
                        "blockNumber": int(receipt['blockNumber'], 16),
                        "transactionHash": receipt['transactionHash'],
                        "logIndex": int(log['logIndex'], 16),
                        "tokenContract": token_contract,
                        "fromAddress": from_address,
                        "toAddress": to_address,
                        "value": str(value) # Store as string to handle large numbers
                    }
                    decoded_transfers.append(transfer_data)
                except Exception as e:
                    # This can happen with non-standard ERC-20 contracts
                    print(f"⚠️ Could not decode a potential transfer log. Error: {e}")

    print(f"✅ Decoded {len(decoded_transfers)} transfer events.")
    return decoded_transfers

In [ ]:
def get_token_metadata(token_address: str) -> Optional[Dict[str, Any]]:
    """
    Fetches ERC-20 token metadata (name, symbol, decimals) using a cache.
    """
    if token_address in TOKEN_METADATA_CACHE:
        return TOKEN_METADATA_CACHE[token_address]
    
    try:
        contract = w3.eth.contract(address=token_address, abi=ERC20_ABI)
        
        metadata = {
            "name": contract.functions.name().call(), 
            "symbol": contract.functions.symbol().call(), 
            "decimals": contract.functions.decimals().call()
            }
        TOKEN_METADATA_CACHE[token_address] = metadata
        return metadata
    except Exception:
        # Some contracts might not be valid ERC-20s or may be proxies.
        TOKEN_METADATA_CACHE[token_address] = None
        return None
    
def enrich_transfers_with_metadata(transfers: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Enriches a list of transfer events with token metadata and calculates the actual value.
    """
    print(f"\nEnriching {len(transfers)} transfers with token metadata...")
    enriched_transfers = []
    for transfer in transfers:
        metadata = get_token_metadata(transfer['tokenContract'])
        if metadata:
            transfer['tokenName'] = metadata['name']
            transfer['tokenSymbol'] = metadata['symbol']

            raw_value_str = transfer.get('rawValue')
            if raw_value_str is not None:
                try:
                    # Calculate the actual amount using the decimals
                    actual_value = int(transfer['rawValue']) / (10 ** metadata['decimals'])
                    transfer['value'] = f"{actual_value:.8f}"
                except (ValueError, TypeError):
                    transfer['value'] = "Error calculating value"
            else:
                transfer['value'] = "Missing rawValue"
                
        else:
            transfer['tokenName'] = "Unknown"
            transfer['tokenSymbol'] = "Unknown"
            transfer['value'] = "Unknown (no metadata)"

        enriched_transfers.append(transfer)
    
    print("✅ Enrichment complete.")
    return enriched_transfers

In [22]:
def parse_and_enrich_transfers(block_data: Dict[str, Any]) -> List[Dict[str, Any]]:
    """Parses, enriches with metadata, and adds USD value to transfers."""
    receipts = block_data.get('receipts', [])
    timestamp = block_data.get('timestamp')
    date_str = datetime.fromtimestamp(timestamp).strftime('%d-%m-%Y')
    
    print(f"\nParsing and enriching {len(receipts)} receipts from {date_str}...")
    enriched_transfers = []

    for receipt in receipts:
        for log in receipt.get('logs', []):
            if log.get('topics') and log['topics'][0] == TRANSFER_EVENT_TOPIC and len(log['topics']) > 2:
                try:
                    token_contract = Web3.to_checksum_address(log['address'])
                    metadata = get_token_metadata(token_contract)
                    if not metadata: continue

                    from_address = Web3.to_checksum_address('0x' + log['topics'][1][-40:])
                    to_address = Web3.to_checksum_address('0x' + log['topics'][2][-40:])
                    
                    log_data = log.get('data', '0x')
                    raw_value = 0 if log_data == '0x' else int(log_data, 16)
                    actual_value = raw_value / (10 ** metadata['decimals'])
                    
                    # Get USD price
                    price = get_historical_price(token_contract, date_str)
                    usd_value = actual_value * price if price is not None else None

                    enriched_transfers.append({
                        "blockNumber": int(receipt['blockNumber'], 16),
                        "transactionHash": receipt['transactionHash'],
                        "logIndex": int(log['logIndex'], 16),
                        "tokenContract": token_contract,
                        "tokenName": metadata['name'],
                        "tokenSymbol": metadata['symbol'],
                        "fromAddress": from_address,
                        "toAddress": to_address,
                        "value": f"{actual_value:.8f}",
                        "usdValue": f"{usd_value:.2f}" if usd_value is not None else "N/A"
                    })
                except Exception as e:
                    print(f"⚠️ Could not process a log. Error: {e}")

    print(f"✅ Fully enriched {len(enriched_transfers)} transfer events.")
    return enriched_transfers


In [23]:
def get_historical_price(token_address: str, date_str: str) -> Optional[float]:
    """Gets historical price for a token on a specific date from CoinGecko, using a cache."""
    coingecko_id = TOKEN_ID_MAP.get(token_address.lower())
    if not coingecko_id: return None

    cache_key = f"{coingecko_id}-{date_str}"
    if cache_key in PRICE_CACHE: return PRICE_CACHE[cache_key]

    try:
        print(f"    Fetching price for {coingecko_id} on {date_str}...")
        url = f"https://api.coingecko.com/api/v3/coins/{coingecko_id}/history?date={date_str}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        price = data.get('market_data', {}).get('current_price', {}).get('usd')
        PRICE_CACHE[cache_key] = price
        time.sleep(1.5) # Respect CoinGecko's free tier rate limit
        return price
    except Exception as e:
        print(f"    ⚠️ Could not fetch price for {coingecko_id}. Error: {e}")
        PRICE_CACHE[cache_key] = None
        return None

Main execution

In [34]:
if __name__ == "__main__":
    try:
        latest_block_number = w3.eth.block_number
        print(f"Latest block on Base: {latest_block_number}")

        block_data = get_block_with_receipts(latest_block_number)

        if block_data:
            enriched_transfers = parse_and_enrich_transfers(block_data)
            
            if enriched_transfers:
                output_filename = f"final_enriched_transfers_block_{latest_block_number}.json"
                with open(output_filename, 'w') as f:
                    json.dump(enriched_transfers, f, indent=2)
                
                print(f"\nSuccessfully saved fully enriched data to '{output_filename}'")
                print("This data is now ready for analysis and storage in a database.")

    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

Latest block on Base: 33647634

Attempting to fetch block and receipts for: 33647634...
✅ Found 244 receipts for block 33647634.

Parsing and enriching 244 receipts from 01-08-2025...
    Fetching price for {'name': 'Wrapped Ether', 'symbol': 'WETH'} on 01-08-2025...
    ⚠️ Could not fetch price for {'name': 'Wrapped Ether', 'symbol': 'WETH'}. Error: 404 Client Error: Not Found for url: https://api.coingecko.com/api/v3/coins/%7B'name':%20'Wrapped%20Ether',%20'symbol':%20'WETH'%7D/history?date=01-08-2025


Exception ignored in: <finalize object at 0x1059fa500; dead>
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/weakref.py", line 575, in __call__
    def __call__(self, _=None):
KeyboardInterrupt: 


✅ Fully enriched 229 transfer events.

Successfully saved fully enriched data to 'final_enriched_transfers_block_33647634.json'
This data is now ready for analysis and storage in a database.
